# Big_Data parsing city

In [1]:
# import libraries
import pandas as pd
import numpy as np
import math

from bs4 import BeautifulSoup as bs
import requests

import urllib.request, urllib.parse, urllib.error
import json
import ssl

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

## Parsing from Wikipedia

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_cities_and_towns_in_Russia_by_population'
soup = bs(requests.get(url).text, 'html.parser')
 
trs = soup.select('table.sortable tr')
 
city_pop_pars = []
 
for tr in trs:
    if tr.td and (tr.select('td[style]')[0].text.rstrip().replace(',', '')).isdigit():
        name = tr.td.text.rstrip()
        population = tr.select('td[style]')[0].text.rstrip().replace(',', '')
        city_pop_pars.append((name, int(population)))

In [3]:
city = pd.DataFrame(city_pop_pars, columns = ['name', 'population'] )

In [4]:
city

,name,population
0,Moscow,12480481
1,Saint Petersburg,5398064
2,Novosibirsk,1625631
3,Yekaterinburg,1493749
4,Kazan,1257391
...,...,...
312,Salekhard,50976
313,Timashyovsk,50792
314,Kirishi,50525
315,Solnechnogorsk,50169


In [5]:
api_key = 42
serviceurl = 'http://py4e-data.dr-chuck.net/json?'

In [6]:
city['lng'] = np.array(city.shape[0])
city['lat'] = np.array(city.shape[0])

In [7]:
for row in range(len(city)):
    address = city['name'][row]

    parms = dict()
    parms['address'] = address
    if api_key is not False: parms['key'] = api_key
    url = serviceurl + urllib.parse.urlencode(parms)

    # print('Retrieving', url)
    uh = urllib.request.urlopen(url, context=ctx)
    data = uh.read().decode()
    # print('Retrieved', len(data), 'characters')

    try:
        js = json.loads(data)
    except:
        js = None

    if not js or 'status' not in js or js['status'] != 'OK':
        print('==== Failure To Retrieve ====')
        print(data)
        continue
        city['lat'][row] = -1
        city['lng'][row] = -1
        print(js['results'][0]['geometry']['location']['lng'], "{}".format(len(js['results'])))
    else:
        city['lat'][row] = js['results'][0]['geometry']['location']['lat']
        city['lng'][row] = js['results'][0]['geometry']['location']['lng']

    # print('lat', lat, 'lng', lng)
    # location = js['results'][0]['formatted_address']
    # print(location)

D:\Programs\Anaconda\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Programs\Anaconda\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


==== Failure To Retrieve ====
{
   "results" : [],
   "status" : "ZERO_RESULTS"
}

==== Failure To Retrieve ====
{
   "results" : [],
   "status" : "ZERO_RESULTS"
}

==== Failure To Retrieve ====
{
   "results" : [],
   "status" : "ZERO_RESULTS"
}

==== Failure To Retrieve ====
{
   "results" : [],
   "status" : "ZERO_RESULTS"
}

==== Failure To Retrieve ====
{
   "results" : [],
   "status" : "ZERO_RESULTS"
}



## Review of Data

In [9]:
city

,name,population,lng,lat
0,Moscow,12480481,37,55
1,Saint Petersburg,5398064,30,59
2,Novosibirsk,1625631,82,54
3,Yekaterinburg,1493749,60,56
4,Kazan,1257391,49,55
...,...,...,...,...
312,Salekhard,50976,66,66
313,Timashyovsk,50792,38,45
314,Kirishi,50525,32,59
315,Solnechnogorsk,50169,36,56


In [10]:
city.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 317 entries, 0 to 316
Data columns (total 4 columns):
name          317 non-null object
population    317 non-null int64
lng           317 non-null int32
lat           317 non-null int32
dtypes: int32(2), int64(1), object(1)
memory usage: 7.6+ KB


In [11]:
# данная API выводит слишком сокрашенные данные 
city[city['name'] == 'Moscow']['lng']

0    37
Name: lng, dtype: int32

In [12]:
city.describe()

,population,lng,lat
count,3.170000e+02,317.000000,317.000000
mean,2.695937e+05,55.470032,56.782334
std,7.915089e+05,48.557644,33.551018
min,5.015400e+04,-122.000000,33.000000
25%,6.634400e+04,38.000000,50.000000
50%,1.055310e+05,45.000000,54.000000
75%,2.400200e+05,60.000000,56.000000
max,1.248048e+07,317.000000,317.000000


In [13]:
city.head()

,name,population,lng,lat
0,Moscow,12480481,37,55
1,Saint Petersburg,5398064,30,59
2,Novosibirsk,1625631,82,54
3,Yekaterinburg,1493749,60,56
4,Kazan,1257391,49,55


In [14]:
city['lng']

0       37
1       30
2       82
3       60
4       49
      ... 
312     66
313     38
314     32
315     36
316    103
Name: lng, Length: 317, dtype: int32

In [15]:
# Вычисление расстояния между двумя точками на сфере
def distance(llat1, llong1, llat2, llong2):
    #pi - число pi, rad - радиус сферы (Земли)
    rad = 6372795

#     #координаты двух точек
#     llat1 = 77.1539
#     llong1 = -120.398

#     llat2 = 77.1804
#     llong2 = 129.55

    #в радианах
    lat1 = llat1*math.pi/180.
    lat2 = llat2*math.pi/180.
    long1 = llong1*math.pi/180.
    long2 = llong2*math.pi/180.

    #косинусы и синусы широт и разницы долгот
    cl1 = math.cos(lat1)
    cl2 = math.cos(lat2)
    sl1 = math.sin(lat1)
    sl2 = math.sin(lat2)
    delta = long2 - long1
    cdelta = math.cos(delta)
    sdelta = math.sin(delta)

    #вычисления длины большого круга
    y = math.sqrt(math.pow(cl2*sdelta,2) + math.pow(cl1*sl2-sl1*cl2*cdelta,2))
    x = sl1*sl2 + cl1*cl2*cdelta
    ad = math.atan2(y,x)
    dist = ad*rad

    return dist

In [16]:
lst = list()

In [17]:
for i in range(len(city)-1):
    for j in range (i + 1, len(city)):
#         dist = ((city['lat'][i] - city['lat'][j])**2 + (city['lng'][i] - city['lng'][j])**2)**(1/2) # вычисление скалярного расстояния
        dist = distance(city['lat'][i],city['lng'][i], city['lat'][j], city['lng'][j]) # вычисление по сфере
        lst.append([ dist, city['name'][i], city['name'][j], city['population'][i], city['population'][j] ])

In [18]:
len (lst)

50086

In [19]:
len(city)

317

In [26]:
res = pd.DataFrame(lst, columns=['dist', 'city_1', 'city_2', 'population_1', 'population_2'] )

In [27]:
res

,dist,city_1,city_2,population_1,population_2
0,6.140087e+05,Moscow,Saint Petersburg,12480481,5398064
1,2.858151e+06,Moscow,Novosibirsk,12480481,1625631
2,1.446497e+06,Moscow,Yekaterinburg,12480481,1493749
3,7.646212e+05,Moscow,Kazan,12480481,1257391
4,4.545845e+05,Moscow,Nizhny Novgorod,12480481,1252236
...,...,...,...,...,...
50081,1.231508e+06,Timashyovsk,Solnechnogorsk,50792,50169
50082,4.657964e+06,Timashyovsk,Cheremkhovo,50792,50154
50083,4.103158e+05,Kirishi,Solnechnogorsk,50525,50169
50084,4.252761e+06,Kirishi,Cheremkhovo,50525,50154


In [28]:
res['dist'].unique()

array([ 614008.67370033, 2858151.45024836, 1446497.40572762, ...,
       2482522.80932432, 4252760.560026  , 4168486.99204907])

In [29]:
# добавляем ограничение на дистанцию
res = res[res['dist'] < 150 ]
res

,dist,city_1,city_2,population_1,population_2
36,0.0,Moscow,Balashikha,12480481,507366
63,0.0,Moscow,Podolsk,12480481,308130
74,0.0,Moscow,Khimki,12480481,259550
80,0.0,Moscow,Mytishchi,12480481,235504
87,0.0,Moscow,Korolyov,12480481,225858
...,...,...,...,...,...
48666,0.0,Polevskoy,Beryozovsky,60979,59030
48699,0.0,Polevskoy,Snezhinsk,60979,51751
48978,0.0,Kumertau,Meleuz,59478,56677
49257,0.0,Beryozovsky,Snezhinsk,59030,51751


<b> Текущая API округляет координаты и тем самым близкие города показывают нулевое расстояние. При подборе нужного API можно добиться точных результатов. </b>

Челленджи:
* Сложность найти общие данные по городам. Были найдены только список городов или сайт, на котором полные данные предоставляются после оплаты. Плохая структурированность Росстата - проще произвести парсинг, чем разобраться в интерфейсе данного сайта
* Неполные геоданные городов.
* По собственной инициативе были взяты города РФ. Решение обусловлено, поскольку компания МТС предлагает свои услуги на территории этой страны

### Saving of results

In [24]:
res.to_csv('city_Salkov_vacancy_BigData.csv')